In [ ]:
import numpy as np
import pandas as pd
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix

import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras import layers, Model
from keras.optimizers import SGD

from sklearn.metrics import classification_report

In [ ]:
image = load_img('test.jpg')
image = img_to_array(image)
image = expand_dims(image, 0)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    shear_range=0.5,
    zoom_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
)
# iterator = datagen.flow(image, batch_size=1)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(15, 25))
axs = axs.ravel()
for i in range(10):
    batch = iterator.next()
    new_image = batch[0].astype('uint8')
    axs[i].imshow(new_image)
plt.show()

In [ ]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
n_classes = 10

y_train = keras.utils.to_categorical(y_train,n_classes)
y_test = keras.utils.to_categorical(y_test,n_classes)

In [ ]:
cat_indices = [i for i, y in enumerate(y_train) if  y[3] == 1]
dog_indices = [i for i, y in enumerate(y_train) if  y[5] == 1]
deleted_cat = np.random.choice(cat_indices, size=4500, replace=False)
deleted_dog = np.random.choice(dog_indices, size=4500, replace=False)
deleted_indices = np.concatenate((deleted_cat, deleted_dog))

y_train = np.delete(y_train, deleted_indices, axis=0)
x_train = np.delete(x_train, deleted_indices, axis=0)

In [ ]:
input_layer = layers.Input (shape=(32,32,3))

conv1 = layers.Conv2D (16, (3, 3), activation='relu', padding='same', strides=1)(input_layer)
conv2 = layers.Conv2D (32, (3, 3), activation='relu', padding='same', strides=1)(conv1)
pool1 = layers.MaxPool2D(pool_size=(2, 2))(conv2)

conv3 = layers.Conv2D (32, (3, 3), activation='relu', padding='same', strides=1)(pool1)
conv4 = layers.Conv2D (64, (3, 3), activation='relu', padding='same', strides=1)(conv3)
pool2 = layers.MaxPool2D(pool_size=(2, 2))(conv4)

conv5 = layers.Conv2D (64, (3, 3), activation='relu', padding='same', strides=1)(pool2)
conv6 = layers.Conv2D (128, (3, 3), activation='relu', padding='same', strides=1)(conv5)
pool3 = layers.MaxPool2D(pool_size=(2, 2))(conv6)

drop1 = layers.Dropout(0.2)(pool3)

flatten = layers.Flatten()(drop1)

fully_connected1 = layers.Dense(300,activation = 'relu')(flatten)
drop2 = layers.Dropout(0.3)(fully_connected1)
fully_connected2 = layers.Dense(100,activation = 'relu')(drop2)
drop3 = layers.Dropout(0.3)(fully_connected2)
fully_connected3 = layers.Dense(10,activation = 'softmax')(drop3)

model = Model(input_layer, fully_connected3)

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, batch_size=128, epochs=50)

In [ ]:
prediction = model.predict(x_test)
prediction = [np.argmax(y) for y in prediction]
labels = [np.argmax(y) for y in y_test]

cm = confusion_matrix(labels, prediction)
sns.heatmap(cm/np.sum(cm), cmap='Blues')

In [ ]:
print(classification_report(labels, prediction))

In [ ]:
cat_indices = [i for i, y in enumerate(y_train) if  y[3] == 1]
dog_indices = [i for i, y in enumerate(y_train) if  y[5] == 1]

cats = x_train[cat_indices]
dogs = x_train[dog_indices]

In [ ]:
def augment_category(datagen, category_data, category_index):
    
    datagen.fit(category_data)
    iterator = datagen.flow(category_data, batch_size=1)
    new_data = []
    for i in range(4500):
        new_data.append(iterator.next()[0])

    label = [0 for _ in range(10)]
    label[category_index] = 1
    labels = [label for sample in new_data]
    
    return new_data, labels

In [ ]:
new_cat, cat_labels = augment_category(datagen, cats, 3)
new_dog, dog_labels = augment_category(datagen, dogs, 5)

In [ ]:
x_train = np.concatenate((x_train, new_cat))
y_train = np.concatenate((y_train, cat_labels))

x_train = np.concatenate((x_train, new_dog))
y_train = np.concatenate((y_train, dog_labels))

In [ ]:
indices = np.array([i for i in range(len(x_train))])
indices = np.random.shuffle(indices)
x_train = x_train[indices].reshape((50000, 32, 32, 3))
y_train = y_train[indices].reshape((50000, 10))

In [ ]:
x_train.shape